In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
import matplotlib.lines as mlines
import ipywidgets as widgets
from ipywidgets import Layout
from math import exp

In [3]:
filename="22BPNODIPS chloroform 600 200mW PTA fixed.csv"#22BPNODIPS chloroform 600 200mW PTA fixed.csv
rawdata = open(filename)

#step 1: split the data from the info at the end
#making it into new files: optional, can stick them into variables instead
fileinfo=filename[0:-4]+"-info"+".csv"
fileinfofile= open(fileinfo, "w") #'w' makes it write only so deletes prev version of it
justdata=filename[0:-4]+"-data"+".csv"
justdatafile=open(justdata, "w")

for line in rawdata:
    if len(line)<100: #if it is not data but text
        fileinfofile.write(line) 
    #step 2: remove the NaNs at the begining
    else: 
        nan=False
        for char in line:
            if char == 'N':
                nan=True
                break
        if nan:
            continue
        else:
            justdatafile.write(line)
fileinfofile.close() #its good practice to close files :)
rawdata.close()
justdatafile.close()
print((open(fileinfo)).read())


Date: March 16, 2015
Sample: NODIPS-22BP-Chloroform
Solvent: THF
Pump energy (uJ): 200 uW
Pump wavelength (nm): 600
Cuvette length (mm): 2
Comments: Time Zero: 505.800 ps
Averaging time: 2.0 s
Number of scans: 2
Measurement time: 00:37:49

Time units: ps
Z axis title: dA


In [4]:
dataarray = (np.genfromtxt(justdata, delimiter=",",  filling_values=0)).T
print(np.shape(dataarray))
X=dataarray[0,1:].T #works
Y=dataarray[1:,0]
Z=dataarray[1:,1:]
print(np.shape(X))
print(np.shape(Y))
print(np.shape(Z))

(456, 454)
(453,)
(455,)
(455, 453)


In [5]:
#step 3: remove the noise at the begining and end
#read the begining, check if its noise, remove it
#same thing from the end

listofR=[]
#print((np.shape(X)))
coef=0.4 #for first dataset it was 0.4 THIS NEEDS ADJUSTING
for col in range((np.shape(X))[0]):
    slice2=Z[:,col]
    R=abs(np.corrcoef(Y,slice2))[0,1]
    listofR.append(R)
    #print(R)
i=0
while listofR[i]<coef: #find uncorrelated data in the beginning 
    Z=np.delete(Z,0,1) #remove that row from Z
    X=np.delete(X,0,0) #remove the wavelength too
    i=i+1
i=(np.shape(X))[0]-1#same thing but from the end
while listofR[i]<coef:
    Z=np.delete(Z,-1,1)
    X=np.delete(X,-1,0)
    i=i-1


In [6]:
#step 4: remove the laser
#add a manual option
fileinforead=open(fileinfo)
pumpwl=0
for line in fileinforead:
    if line[0:15]=='Pump wavelength':
        pumpwl= int(line[-4:-1])
        print(pumpwl)
if pumpwl!=0:
    starti = np.searchsorted(X, (pumpwl - 20))#this can be adjusted
    endi = np.searchsorted(X, (pumpwl + 20))
    Z[:,starti:endi]=0
if pumpwl==0:
    print("did not find data")

600


In [7]:
%matplotlib widget
plt.pcolormesh(X,Y,Z, cmap='jet', shading='nearest')
#shading='nearest' makes the pixels centered on the coordinates
#plt.yscale('log')

plt.ylabel('Time(ps)')
plt.xlabel('Wavelength(nm)')

plt.savefig("mygraph.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
#DONT need this :)
#making the curve 
x=[]
y=[]
for i in range(0,10):
    x.append(400+i*50)    
    y.append(input("Y value for wavelength "+str(x[i])+":"))

Y value for wavelength 400:-0.65
Y value for wavelength 450:-0.2


In [ ]:
#finds the important wavelengths for the gaussian
maxZ=max(Z)
fiveper = maxZ*0.05

for i in range(0,10):
    stArt=np.searchsorted(X,(400+i*50))
    eNd=np.searchsorted(X,(450+i*50))
    #also make the list of x's
    #^make that better.
    for wl in range(stArt,eNd):
        maxes.append(max(Z[:,wl]))#finds max of each of 50 wls, appends to maxes[]
    if max(maxes)< fiveper:
        continue
    else:
        wl= maxes.index(max(maxes))+stArt #finds index of max(maxes), adds to start of chunk
        #do the gauss thing for this wl
        Finalt0.append[0]#append the real finalt0
    maxes=[]  



In [9]:
#CHIRP CORRECTOR-------
it0=[]
for wl in range(0,np.shape(X)[0]):
    it0.append(np.searchsorted(Y,t0[wl]))#gets the index of that wl's t0
    
T0=float(y[0])#-0.6
iT0=np.searchsorted(Y,T0)
Znew=Z.copy()
Ynew=Y.copy()
for wl in range(0,np.shape(X)[0]):
    shift=0-t0[wl] #note, not all data is around zero, might have to change
    for t in range(iT0,np.shape(Y)[0]-1):#start changing from main T0
        Ynew[t]=Y[t]+shift
        Znew[t,wl]=Z[t,wl]
    f=interp1d(Ynew,Znew[:,wl], kind="linear", fill_value="extrapolate")       
    Znew[:,wl]=f(Y)

In [87]:
#gaussian/ chirp corrector 2.0
e=2.718281828
def curve1(t, y0,pw,t0,a1,k1,a2,k2,a3,k3):#,a4,k4,a5,k5,a6,k6):
    return y0+0.5*a1*(e**(-k1*(t-t0-(pw**2)*k1/2)))*(1+erf((t-t0-k1*(pw**2))/(1.41421356237*pw)))+0.5*(a2*e**(-k2*(t-t0-(pw**2)*k2/2)))*(1+erf((t-t0-k2*(pw**2))/(1.41421356237*pw)))+0.5*(a3*e**(-k3*(t-t0-(pw**2)*k3/2)))*(1+erf((t-t0-k3*(pw**2))/(1.41421356237*pw)))#+0.5*pow(a4,(-k4*(t-t0-pow(pw,2)*k4/2)))*(1+erf((t-t0-k4*pow(pw,2))/(1.41421356237*pw)))+0.5*pow(a5,(-k5*(t-t0-pow(pw,2)*k5/2)))*(1+erf((t-t0-k5*pow(pw,2))/(1.41421356237*pw)))+0.5*pow(a6,(-k6*(t-t0-pow(pw,2)*k6/2)))*(1+erf((t-t0-k6*pow(pw,2))/(1.41421356237*pw)))
T=[]
for wl in range(0,np.shape(x)[0]):
    
    f=interp1d(Y,Z[:,wl], kind="linear", fill_value="extrapolate")   
    Ybig=np.linspace(-100,2600,20000) #start,end,num of points
    Zbig=f(Ybig)
    
    if starti<=wl<=endi: #these are the indices around the pump wavelength that are all zero
        T.append(0) #whyyyy does this only kind of work...
        continue
    params, pc = curve_fit(curve1, Y,Z[:,wl],maxfev=100000, p0=[1e-6,0.1,params[2],1e-3,1,1e-3,1e-2,1e-3,1e-3])
    T.append(params[2]) #use this as next
    print(T[wl],X[wl])
print("done!")

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in power
  after removing the cwd from sys.path.


-0.2566705092515921 407.122296725
-0.2278101863929131 408.290695225
-0.2548170137564285 409.459231885
-0.2681663664704614 410.627906705
-0.2701122371544788 411.796719685
-0.27826489911365143 412.965670825
-0.2716227152092423 414.134760125
-0.2523886010059614 415.303987585
-0.22339514025946086 416.473353205
-0.23661178155858822 417.642856985
-0.22537952748445161 418.812498925
-0.21617979473620877 419.982279025
-0.21002546190581226 421.152197285
-0.19289019639991978 422.322253705
-0.18884495428562337 423.492448285
-0.18575117285347556 424.662781025
-0.17163909905920613 425.833251925
-0.1692104774791442 427.003860985
-0.15965186914861726 428.174608205
-0.15185534160330488 429.345493585
-0.14462632797541827 430.516517125
-0.13437524143581556 431.687678825
-0.12670581557885954 432.858978685
-0.119797400049032 434.030416705
-0.11181648212329214 435.201992885
-0.10538241607614389 436.373707225
-0.09847235939859945 437.545559725
-0.08914085003013057 438.717550385
-0.08350418619462559 439.88967

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in multiply
  after removing the cwd from sys.path.


0.1816729558901211 491.600125685
0.18898753380089245 492.778471705
0.1931310433263053 493.956955885
0.20082647567574083 495.135578225
0.20674745071441814 496.314338725
0.21409693047011297 497.493237385
0.21977005157118168 498.672274205
0.2241528710915008 499.851449185
0.22917490143680772 501.030762325
0.2329885312267526 502.210213625
0.2407532172198933 503.389803085
0.23983666134324216 504.569530705
0.2419843852321839 505.749396485
0.2460359183700653 506.929400425
0.24735733680785155 508.109542525
0.25183281883784514 509.289822785


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in add
  after removing the cwd from sys.path.


0.25750359757110575 510.470241205
0.259508855981312 511.650797785
0.2647357839408938 512.831492525
0.2716384456066258 514.012325425
0.2750327592323424 515.193296485
0.2807919019275372 516.374405705
0.28496299981243495 517.555653085
0.2901786002805635 518.737038625
0.2933320932444332 519.918562325
0.29752103402959995 521.100224185
0.30074366885227927 522.282024205
0.30590596776787066 523.463962385
0.3095227137799555 524.646038725
0.31479524815156157 525.828253225
0.3167599392033457 527.010605885
0.32090379439532424 528.193096705
0.3244367036273091 529.375725685
0.32670903475670077 530.558492825
0.3303828080294171 531.741398125
0.33317423148148867 532.924441585
0.3371537069494268 534.107623205
0.34078113707138935 535.290942985
0.3428062427829022 536.474400925
0.34632489004673417 537.657997025
0.3498051545537653 538.841731285
0.3511601649302056 540.025603705
0.353467565356205 541.209614285
0.3567757678595757 542.393763025
0.3595179407829646 543.578049925
0.3609601938100197 544.762474985
0

In [8]:
%matplotlib widget
#interpolates your points to a curve 
x=[400,450,500,550,600,650,700,750,800,850]
y=['-0.65', '-0.2', '0.1', '0.25', '0.45', '0.5', '0.55', '0.6', '0.6',"0.65"]


#curve_fit need a function.
def test_func(x, a, b, c,d):
    return a*pow(x,3)+b*pow(x,2)+c*x+d
params, pc = curve_fit(test_func, x, y, maxfev=6000)
print(params)
t0=test_func(X, params[0], params[1], params[2],params[3])

plt.pcolormesh(X,Y,Z, cmap='jet', shading='nearest')
plt.plot(X,t0, color="white")
plt.ylim(-1,1)
plt.colorbar()

[ 2.48640198e-08 -5.58624612e-05  4.22950602e-02 -1.02078310e+01]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
#also dont think i need this
#CHIRP CORRECTOR 2.0??-------
Znewer=Z.copy()
Ynewer=Y.copy()

for wl in range(0,np.shape(X)[0]):
    shift=0-T[wl] #note, not all data is around zero, might have to change
    for t in range(0,np.shape(Y)[0]-1):#start changing from main T0
        Ynewer[t]=Y[t]+shift
        Znewer[t,wl]=Z[t,wl]
    f=interp1d(Ynewer,Znewer[:,wl], kind="linear", fill_value="extrapolate")       
    Znewer[:,wl]=f(Y)
print('done!')

done!


In [89]:
%matplotlib widget
fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.pcolormesh(X,Y,Z, cmap='jet', shading='nearest') #before
ax1.axis(ymin=-1,ymax=1)
ax2.pcolormesh(X,Y,Znewer, cmap='jet', shading='nearest')#manual curve
ax2.axis(ymin=-1,ymax=1)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(406.53809747500003, 844.593958375, -1.0, 1.0)

In [56]:
filename="testdata.csv"#22BPNODIPS chloroform 600 200mW PTA fixed.csv
rawdata = open(filename)

#step 1: split the data from the info at the end
#making it into new files: optional, can stick them into variables instead
fileinfo=filename[0:-4]+"-info"+".csv"
fileinfofile= open(fileinfo, "w") #'w' makes it write only so deletes prev version of it
justdata=filename[0:-4]+"-data"+".csv"
justdatafile=open(justdata, "w")

for line in rawdata:
    if len(line)<100: #if it is not data but text
        fileinfofile.write(line) 
    #step 2: remove the NaNs at the begining
    else: 
        nan=False
        for char in line:
            if char == 'N':
                nan=True
                break
        if nan:
            continue
        else:
            justdatafile.write(line)
fileinfofile.close() #its good practice to close files :)
rawdata.close()
justdatafile.close()
print((open(fileinfo)).read())
dataarray1 = (np.genfromtxt(justdata, delimiter=",",  filling_values=0)).T
print(np.shape(dataarray1))
A=dataarray1[0,1:].T #works
B=dataarray1[1:,0]
C=dataarray1[1:,1:]


Pump Wavelength (nm): 605Pump Wavelength (nm): 605
(454, 337)


In [15]:
print(np.shape(Znew))

(455, 367)


In [58]:
%matplotlib widget
fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.pcolormesh(X,Y,Znewer, cmap='jet', shading='nearest')
ax1.axis(ymin=-1,ymax=1)
ax2.pcolormesh(A,B,C, cmap='jet', shading='nearest')
ax2.axis(ymin=-1,ymax=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(415.888615, 818.418, -1.0, 1.0)

In [38]:
#thing to check how different i am from the properly fixed one
%matplotlib widget

D=Znew[:453,8:344]-C #datasets are different sizes, this centers them
for wl in range(0,np.shape(C)[1]): 
    for t in range(0,np.shape(C)[0]):
        D[t,wl]=D[t,wl]-C[t,wl]

plt.pcolormesh(X[:336],Y[:453],D, cmap='jet', shading='nearest')
plt.ylim(0,10)
plt.colorbar()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
%matplotlib widget
fig, ax = plt.subplots()


def slider(t):
    #finds closest index to that t
    i = np.searchsorted(Y, t)
    #get horizontal slice
    slice1=Z[i,:]
    [l.remove() for l in ax.lines]
    ax.plot(X,slice1,color="black")
   
    
widgets.interact(slider, t=(-5,2500,3))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1246, description='t', max=2500, min=-5, step=3), Output()), _dom_classe…

<function __main__.slider(t)>

In [12]:
#VERTICAL SLICE----------
%matplotlib widget
fig, ax = plt.subplots()


def slider(wl):
    #finds closest index to that wl
    i = np.searchsorted(X, wl)
    print(i)
    slice2=Z[:,i];
    [l.remove() for l in ax.lines]
    ax.plot(Y,slice2,color="black")

#Slider used to select wavelength
widgets.interact(slider, wl=(400,700,3))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=550, description='wl', max=700, min=400, step=3), Output()), _dom_classe…

<function __main__.slider(wl)>

In [ ]:
#error function....
#define parameters of the chunk of plot we r looking at
#loop through all vertical slices and do the error function thing
#use those points as the curve
#stretch/interpolate
start=np.searchsorted(Y, 0.6)
end=np.searchsorted(Y, -0.6)
wl=np.searchsorted(X, 528)
error=erf(Z[start:end,wl])
print(error)


In [80]:
np.float128(1) + np.float128(2**-64) - np.float128(1)
#an answer of 0.0 indicates that the float128 type contains no more than 64 bits of precision

0.0

In [22]:
#Finds Gaussian convoluted with 3 exponentials at this wavelength
%matplotlib widget
plt.plot(Y,Z[:,248]) #blue wl=700
plt.plot(Ybig,Zbig) #orange
ExpGauss2=curve1(Ybig,params[0],params[1],params[2],params[3],params[4],params[5],params[6],params[7],params[8])#,params[9],params[10],params[11],params[12])
plt.plot(Ybig,ExpGauss2)#green
delta2=Zbig-ExpGauss2
plt.plot(Ybig,delta2)#red
#[-3.36189614e-05  1.97442342e+00 -5.70929342e+03  1.00000217e+00
# 1.52916609e+03  1.00000220e+00  1.49084208e+03  1.00000168e+00
#  1.46483839e+03]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [81]:
#saves everything
fixeddata =np.insert(Znew,0,Y, axis=1)
Xx= np.insert(X,0,0,axis=0)
Fixeddata= np.insert(fixeddata,0,Xx,axis=0)
np.savetxt(justdata, Fixeddata, delimiter=",")